In [1]:
import cv2
import os
from imgaug import augmenters as iaa
from skimage import color

import matplotlib.pyplot as plt
from PIL import Image

In [15]:
# cara cek format gambar
image = cv2.imread('dataset_tanpa_augmentasi/RingSpot/RingSpot(120).jpg')
img = Image.fromarray(image)
img.mode

'RGB'

In [27]:
folder_input = 'dataset_tanpa_augmentasi' # folder input
folder_output = 'dataset_aug' # folder output

dataset_images = [dataset_image for dataset_image in os.listdir(folder_input)]

for name_folder in dataset_images:
    os.makedirs(os.path.join(folder_output, name_folder), exist_ok=True)


# augmentasi gambar
augmenter = iaa.Sequential([
    iaa.Fliplr(0.5),
    # iaa.Crop(percent=(0, 0.1)),
    iaa.Affine(
        rotate=(-45, 45),
        shear=(-16, 16)
    ),
    iaa.GaussianBlur(sigma=(0, 0.5)),
    iaa.LinearContrast((0.75, 1.5)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5)
], random_order=True)

def augment_image(img):
    return augmenter.augment_image(img)

# fungsi untuk menentukan nama label dari nama file
def get_name_label(filename):
    for labels in dataset_images:
        if labels in filename:
            return labels
    return None


# memproses semua gambar
for new_file in os.listdir(folder_input):
    path_imgs = os.path.join(folder_input, new_file)
    label = get_name_label(new_file)
    for path_img in os.listdir(path_imgs):
        original_path = os.path.join(path_imgs, path_img)
        image = cv2.imread(original_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # pastikan file memiliki ekstensi
        _, ext = os.path.splitext(original_path)
        if ext.lower() in [".jpg", ".jpeg", ".png"]:
            output_path = os.path.join(folder_output, label, path_img)
            # print(output_path)
            cv2.imwrite(output_path, image)
        else:
            print(f"⚠️ Ekstensi tidak dikenali: {path_img}")

        for i in range(3):
            img_aug = augment_image(image)
            new_filename = f"{path_img.split('.')[0]}_aug{i+1}.jpg"
            save_path = os.path.join(folder_output, label, new_filename)
            cv2.imwrite(save_path, img_aug)

In [28]:
# img_lokasi = imread(file_path[0])

# plt.figure(figsize=(8, 6))
# plt.imshow(img_lokasi)
# plt.show()